In [219]:
import sqlite3, shutil

In [220]:
shutil.copy("openstudio_refrigeration_system.db", "openstudio_refrigeration_system_wk.db")
db_path="openstudio_refrigeration_system_wk.db"

##### Select Cases and walk-in units

In [227]:
def calculate_cooling_capacity(unit_rated_capacity, unit_length, number_of_units):
    # calculate total rated cooling capacity per each case type
    if unit_rated_capacity <= 0:
        raise ValueError(f"Invalid unit rated capacity: {unit_rated_capacity}")

    if unit_length <= 0:
        raise ValueError(f"Invalid unit length: {unit_length}")

    if number_of_units <= 0:
        raise ValueError(f"Invalid number of units: {number_of_units}")

    return unit_rated_capacity * unit_length * number_of_units

def get_data_from_db(db_path, selected_case_units, selected_walkin_units):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    case_data = {}
    walkin_data = {}

    if selected_case_units:
        placeholders = ', '.join(['?'] * len(selected_case_units))
        query = f"SELECT case_name, template, operation_type, rated_capacity, unit_length FROM refrigeration_cases WHERE case_name IN ({placeholders})"
        cursor.execute(query, list(selected_case_units.keys()))
        
        for case_name, template, operation_type, rated_capacity, unit_length in cursor.fetchall():
            unit_count = selected_case_units.get(case_name, 0)
            total_rated_capacity = calculate_cooling_capacity(rated_capacity, unit_length, unit_count)
            case_data[case_name] = {
                "template": template,
                "operation_type": operation_type,
                "rated_capacity": rated_capacity,
                "unit_length": unit_length,
                "unit_count": unit_count,
                "total_rated_capacity": total_rated_capacity
            }

    if selected_walkin_units:
        placeholders = ', '.join(['?'] * len(selected_walkin_units))
        query = f"SELECT walkin_name, template, operation_type, rated_capacity FROM refrigeration_walkins WHERE walkin_name IN ({placeholders})"
        cursor.execute(query, list(selected_walkin_units))

        for walkin_name, template, operation_type, rated_capacity in cursor.fetchall():
            walkin_data[walkin_name] = {
                "template": template,
                "operation_type": operation_type,
                "rated_capacity": rated_capacity
            }

    conn.close()
    return case_data, walkin_data

#### TEST

In [228]:
selected_case_units = {
    "Old LT Coffin - Ice Cream": 2,
    "Old LT Reach-in - Frozen Food": 1,
    "Old MT Vertical Open - All":5
}

selected_walkin_units = {
    "Old MT Walk-in Cooler - 120SF with no glass door",
    "Old LT Walk-in Freezer - 480SF"
}


case_data = get_case_data_from_db(db_path, selected_case_units)
print("Case Data:")
print(case_data)


walkin_data = get_walkin_data_from_db(db_path, selected_walkin_units)
print(walkin_data)

Case Data:
{'Old LT Coffin - Ice Cream': {'template': 'old', 'operation_type': 'LT', 'rated_capacity': 695.2, 'unit_length': 2.0, 'unit_count': 2, 'total_rated_capacity': 2780.8}, 'Old LT Reach-in - Frozen Food': {'template': 'old', 'operation_type': 'LT', 'rated_capacity': 583.7, 'unit_length': 3.0, 'unit_count': 1, 'total_rated_capacity': 1751.1000000000001}, 'Old MT Vertical Open - All': {'template': 'old', 'operation_type': 'MT', 'rated_capacity': 1437.6, 'unit_length': 2.4, 'unit_count': 5, 'total_rated_capacity': 17251.199999999997}}
{'Old LT Walk-in Freezer - 480SF': {'template': 'old', 'operation_type': 'LT', 'rated_capacity': 7725.5}, 'Old MT Walk-in Cooler - 120SF with no glass door': {'template': 'old', 'operation_type': 'MT', 'rated_capacity': 3313.2}}


#### Allocate cases to each rack

In [239]:
def assign_racks_to_cases_and_walkins(db_path, selected_case_units, selected_walkin_units, max_capacity_per_rack=30000):
    # get case and walkin data from DB
    case_data, walkin_data = get_data_from_db(db_path, selected_case_units, selected_walkin_units)
    
    # assign cases and walkins to MT rack and LT rack
    mt_racks = []
    lt_racks = []

    def distribute_units(data, racks, max_capacity_per_rack, is_walkin=False):
        units = sorted(
            data.items(),
            key=lambda x: x[1]['rated_capacity'] if is_walkin else x[1].get('total_rated_capacity', 0),
            reverse=True
        )

        current_rack = []
        current_capacity = 0

        for name, item in units:
            # If it's a walkin, use rated_capacity, else use total_rated_capacity for cases
            if is_walkin:
                total_capacity = item['rated_capacity']
            else:
                total_capacity = item.get('total_rated_capacity', item['rated_capacity'])

            # If adding this unit to the current rack does not exceed the max capacity, add it
            if current_capacity + total_capacity <= max_capacity_per_rack:
                current_rack.append({'name': name, 'capacity': total_capacity})
                current_capacity += total_capacity
            else:
                if current_rack:
                    racks.append(current_rack)
                current_rack = [{'name': name, 'capacity': total_capacity}]
                current_capacity = total_capacity

        if current_rack:
            racks.append(current_rack)

    # Assign to MT rack, combining case and walkin units together
    mt_data = {**{name: case for name, case in case_data.items() if 'MT' in name}, **{name: walkin for name, walkin in walkin_data.items() if 'MT' in name}}
    distribute_units(mt_data, mt_racks, max_capacity_per_rack)

    # Assign to LT rack, combining case and walkin units together
    lt_data = {**{name: case for name, case in case_data.items() if 'LT' in name}, **{name: walkin for name, walkin in walkin_data.items() if 'LT' in name}}
    distribute_units(lt_data, lt_racks, max_capacity_per_rack)
    
    return mt_racks, lt_racks

def display_rack_capacity(racks):
    for i, rack in enumerate(racks, 1):
        total_capacity = sum(item['capacity'] for item in rack)
        print(f"Rack {i}: Total Capacity = {total_capacity} W")
        for item in rack:
            print(f"  - {item['name']} : {item['capacity']} W")
        print()

#### TEST

In [240]:
selected_case_units = {
    "Old LT Coffin - Ice Cream": 2,
    "Old LT Reach-in - Frozen Food": 1,
    "Old MT Vertical Open - All":5
}

selected_walkin_units = {
    "Old MT Walk-in Cooler - 120SF with no glass door",
    "Old LT Walk-in Freezer - 480SF"
}


case_data = get_case_data_from_db(db_path, selected_case_units)
print("Case Data:")
print(case_data)


walkin_data = get_walkin_data_from_db(db_path, selected_walkin_units)
print(walkin_data)

Case Data:
{'Old LT Coffin - Ice Cream': {'template': 'old', 'operation_type': 'LT', 'rated_capacity': 695.2, 'unit_length': 2.0, 'unit_count': 2, 'total_rated_capacity': 2780.8}, 'Old LT Reach-in - Frozen Food': {'template': 'old', 'operation_type': 'LT', 'rated_capacity': 583.7, 'unit_length': 3.0, 'unit_count': 1, 'total_rated_capacity': 1751.1000000000001}, 'Old MT Vertical Open - All': {'template': 'old', 'operation_type': 'MT', 'rated_capacity': 1437.6, 'unit_length': 2.4, 'unit_count': 5, 'total_rated_capacity': 17251.199999999997}}
{'Old LT Walk-in Freezer - 480SF': {'template': 'old', 'operation_type': 'LT', 'rated_capacity': 7725.5}, 'Old MT Walk-in Cooler - 120SF with no glass door': {'template': 'old', 'operation_type': 'MT', 'rated_capacity': 3313.2}}


In [241]:
mt_racks, lt_racks = assign_racks_to_cases_and_walkins(db_path, selected_case_units, selected_walkin_units)


print("MT Racks:")
display_rack_capacity(mt_racks)

print("LT Racks:")
display_rack_capacity(lt_racks)

MT Racks:
Rack 1: Total Capacity = 20564.399999999998 W
  - Old MT Vertical Open - All : 17251.199999999997 W
  - Old MT Walk-in Cooler - 120SF with no glass door : 3313.2 W

LT Racks:
Rack 1: Total Capacity = 12257.4 W
  - Old LT Coffin - Ice Cream : 2780.8 W
  - Old LT Reach-in - Frozen Food : 1751.1000000000001 W
  - Old LT Walk-in Freezer - 480SF : 7725.5 W



In [242]:
selected_case_units = {
    "Old LT Coffin - Ice Cream": 3,
    "Old LT Reach-in - Frozen Food": 3,
    "Old MT Vertical Open - All":10,
    "Old MT Service - Meat Deli Bakery":4
}

selected_walkin_units = {
    "Old MT Walk-in Cooler - 120SF with no glass door",
    "Old LT Walk-in Freezer - 480SF",
    "Old MT Walk-in Cooler - 64SF with glass door"
}


case_data = get_case_data_from_db(db_path, selected_case_units)
print("Case Data:")
print(case_data)


walkin_data = get_walkin_data_from_db(db_path, selected_walkin_units)
print(walkin_data)
mt_racks, lt_racks = assign_racks_to_cases_and_walkins(db_path, selected_case_units, selected_walkin_units)


print("MT Racks:")
display_rack_capacity(mt_racks)

print("LT Racks:")
display_rack_capacity(lt_racks)

Case Data:
{'Old LT Coffin - Ice Cream': {'template': 'old', 'operation_type': 'LT', 'rated_capacity': 695.2, 'unit_length': 2.0, 'unit_count': 3, 'total_rated_capacity': 4171.200000000001}, 'Old LT Reach-in - Frozen Food': {'template': 'old', 'operation_type': 'LT', 'rated_capacity': 583.7, 'unit_length': 3.0, 'unit_count': 3, 'total_rated_capacity': 5253.3}, 'Old MT Service - Meat Deli Bakery': {'template': 'old', 'operation_type': 'MT', 'rated_capacity': 808.0, 'unit_length': 2.4, 'unit_count': 4, 'total_rated_capacity': 7756.799999999999}, 'Old MT Vertical Open - All': {'template': 'old', 'operation_type': 'MT', 'rated_capacity': 1437.6, 'unit_length': 2.4, 'unit_count': 10, 'total_rated_capacity': 34502.399999999994}}
{'Old LT Walk-in Freezer - 480SF': {'template': 'old', 'operation_type': 'LT', 'rated_capacity': 7725.5}, 'Old MT Walk-in Cooler - 120SF with no glass door': {'template': 'old', 'operation_type': 'MT', 'rated_capacity': 3313.2}, 'Old MT Walk-in Cooler - 64SF with gla